In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [13]:
# 연도별 추출을 위해 시작연도와 마지막연도를 변수로 가지는 함수 생성
def game(start, end):
    game_list = []
    base_url = 'https://www.igdb.com'
    
# 장르 번역작업 및 통합을 위한 리스트
    tran_eng = ['fighting', 'shooter', 'music', 'platform', 'puzzle', 'racing', 'real-time-strategy-rts', 'role-playing-rpg', 'simulator', 'sport', 'strategy', 'turn-based-strategy-tbs', 'tactical', 'quiz-trivia', 'hack-and-slash-beat-em-up', 'adventure', 'arcade', 'indie', 'card-and-board-game']
    tran_kor = ['격투', '슈팅', '리듬', '플랫폼', '퍼즐', '레이싱', '전략', 'RPG', '시뮬레이션', '스포츠', '전략', '전략', '전략', '퍼즐', '액션', '어드벤쳐', '아케이드', '인디', '보드']
    
# 연도에 따른 게임리스트 추출작업
    for year in range(start, end+1):
        url = 'https://www.igdb.com/top-100/games/platform/all/'+str(year)
        response = requests.get(url)
        html_str = response.text
        soup = BeautifulSoup(html_str, 'html.parser')
    
# selector의 숫자를 변경시켜가며 데이터 추출
        num = 0
        while True:
            num += 1
            selector = f'#games-top-list > tr:nth-child({num})'
            a_list = soup.select(selector)
            if not a_list : break
            title = a_list[0].select_one('a')['href'][7:].replace('-', ' ')
            year = int(a_list[0].select('span')[0].text[1:5])
            url = base_url + a_list[0].select_one('a')['href']
            rate = int(a_list[0].select('span')[-2].text)
            game_list.append([title,year,url,rate])

# 게임장르가 다른 url에 존재하므로 url정보를 바꿔서 장르 추출
    for list in game_list:
        url = list[2]
        response = requests.get(url)
        html_str = response.text
        soup = BeautifulSoup(html_str, 'html.parser')
        b_list = soup.find_all(class_='gamepage-tabs')
        for i in b_list[0].select('a'):
            if str(i)[3:7] == 'data':
                if i['href'][1:7] == 'genres':
                    genre = i['href'][8:]
                    if genre in tran_eng:   # 장르 번역 작업
                        genre = tran_kor[tran_eng.index(genre)]
                    if genre not in list:  # 장르 중복 제외 작업
                        list.append(genre)

# 게임마다 장르의 수가 다르기에 장르탭의 갯수를 맞추는 작업
    maxlen = 0
    for item1 in game_list:  # 가장 많은 장르를 가진 게임의 장르 수 구하기
        if len(item1)-4 > maxlen :
            maxlen = len(item1)-4
    for item2 in game_list:  # 리스트의 길이를 맞추기 위해 장르 최댓값보다 적은 리스트에 None 추가
        if len(item2)-4 < maxlen:
            for j in range(maxlen-len(item2)+4):
                item2.append(None)

# 장르의 최대수만큼 frame에 장르탭 추가    
    frame = ['게임이름', '출시연도', 'url', '평점']
    for k in range(maxlen):
        frame.append('장르'+str(k+1))
    
# 데이터프레임화 및 csv로 추출    
    df = pd.DataFrame(game_list, columns=frame)
    df.to_csv('project/console_eng_data.csv', index=False)
    return df
game(2011,2021)

,게임이름,출시연도,url,평점,장르1,장르2,장르3,장르4,장르5,장르6
0,the legend of zelda ocarina of time 3d,2011,https://www.igdb.com/games/the-legend-of-zelda...,92,어드벤쳐,플랫폼,퍼즐,RPG,None,None
1,portal 2,2011,https://www.igdb.com/games/portal-2,92,어드벤쳐,플랫폼,퍼즐,슈팅,None,None
2,resident evil 4 hd,2011,https://www.igdb.com/games/resident-evil-4-hd,89,어드벤쳐,슈팅,None,None,None,None
3,gemini rue,2011,https://www.igdb.com/games/gemini-rue,88,어드벤쳐,인디,point-and-click,None,None,None
4,cave story plus 1,2011,https://www.igdb.com/games/cave-story-plus--1,88,어드벤쳐,인디,플랫폼,슈팅,None,None
...,...,...,...,...,...,...,...,...,...,...
760,minecraft dungeons,2020,https://www.igdb.com/games/minecraft-dungeons,71,어드벤쳐,액션,RPG,None,None,None
761,it takes two,2021,https://www.igdb.com/games/it-takes-two,93,어드벤쳐,플랫폼,None,None,None,None
762,valheim,2021,https://www.igdb.com/games/valheim,89,어드벤쳐,인디,RPG,None,None,None
763,resident evil village,2021,https://www.igdb.com/games/resident-evil-village,83,어드벤쳐,None,None,None,None,None
